In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
df=pd.read_csv('smote.csv',na_values=-1)
cut=df.iloc[0:50000,:]
cut.isnull().sum()
y=cut['target']
cut.drop(columns='target',inplace=True)

In [3]:
missing_values=pd.DataFrame({'columns':cut.columns,'values':cut.isnull().sum(),
                             'percentage':cut.isnull().sum()/cut.shape[0]*100})
missing_values.sort_values(by='percentage',ascending=False)
cols_to_drop=list(missing_values[missing_values['percentage']>40]['columns'])
#cut.drop(columns=cols_to_drop,inplace=True)

In [4]:
cut.drop(columns=cols_to_drop,inplace=True)
cols_with_no_na=list(missing_values[missing_values['percentage']==0]['columns'])
a=[]
for col in cols_with_no_na:
    if len(np.unique(cut[col]))==1 or len(np.unique(cut[col]))==cut.shape[0]:
        a.append(col)
cut.drop(columns=a,inplace=True)
cut.shape

(50000, 55)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cut, y, test_size=0.3, random_state=42)
X_train.shape
cat_cols=[col for col in X_train.columns if 'cat' in col or 'bin' in col ]
num_cols=[col for col in X_train.columns if 'cat' not in col or 'bin' not in col ]
num_cols
for col in num_cols:
    X_train[col]=X_train[col].fillna(X_train[col].mean())
    X_test[col]=X_test[col].fillna(X_train[col].mean())
for col in cat_cols:
    X_train[col]=X_train[col].fillna(X_train[col].mode()[0]).astype('object')
    X_test[col]=X_test[col].fillna(X_train[col].mode()[0]).astype('object')
X_train.dtypes

ps_ind_01           int64
ps_ind_02_cat      object
ps_ind_03           int64
ps_ind_04_cat      object
ps_ind_05_cat      object
ps_ind_06_bin      object
ps_ind_07_bin      object
ps_ind_08_bin      object
ps_ind_09_bin      object
ps_ind_10_bin      object
ps_ind_11_bin      object
ps_ind_12_bin      object
ps_ind_13_bin      object
ps_ind_14           int64
ps_ind_15           int64
ps_ind_16_bin      object
ps_ind_17_bin      object
ps_ind_18_bin      object
ps_reg_01         float64
ps_reg_02         float64
ps_reg_03         float64
ps_car_01_cat      object
ps_car_02_cat      object
ps_car_04_cat      object
ps_car_06_cat      object
ps_car_07_cat      object
ps_car_08_cat      object
ps_car_09_cat      object
ps_car_10_cat      object
ps_car_11_cat      object
ps_car_11         float64
ps_car_12         float64
ps_car_13         float64
ps_car_14         float64
ps_car_15         float64
ps_calc_01        float64
ps_calc_02        float64
ps_calc_03        float64
ps_calc_04  

In [6]:
from sklearn.preprocessing import LabelEncoder
import numpy as np


class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)

In [7]:
label=LabelEncoderExt()
for col in cat_cols:
    label.fit(X_train[col])
    X_train[col]=label.transform(X_train[col]).astype('object')
    X_test[col]=label.transform(X_test[col]).astype('object')


In [8]:
X_train.dtypes

ps_ind_01           int64
ps_ind_02_cat      object
ps_ind_03           int64
ps_ind_04_cat      object
ps_ind_05_cat      object
ps_ind_06_bin      object
ps_ind_07_bin      object
ps_ind_08_bin      object
ps_ind_09_bin      object
ps_ind_10_bin      object
ps_ind_11_bin      object
ps_ind_12_bin      object
ps_ind_13_bin      object
ps_ind_14           int64
ps_ind_15           int64
ps_ind_16_bin      object
ps_ind_17_bin      object
ps_ind_18_bin      object
ps_reg_01         float64
ps_reg_02         float64
ps_reg_03         float64
ps_car_01_cat      object
ps_car_02_cat      object
ps_car_04_cat      object
ps_car_06_cat      object
ps_car_07_cat      object
ps_car_08_cat      object
ps_car_09_cat      object
ps_car_10_cat      object
ps_car_11_cat      object
ps_car_11         float64
ps_car_12         float64
ps_car_13         float64
ps_car_14         float64
ps_car_15         float64
ps_calc_01        float64
ps_calc_02        float64
ps_calc_03        float64
ps_calc_04  

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,roc_auc_score,roc_curve
regression=LogisticRegression()
regression.fit(X_train,y_train)
regression_predict=regression.predict(X_test)
confusion_matrix(y_test,regression_predict)
print('accuarcy_score',accuracy_score(y_test,regression_predict))
print('recall',recall_score(y_test,regression_predict))

print('precision:',precision_score(y_test,regression_predict))


accuarcy_score 0.9630666666666666
recall 0.0
precision: 0.0


In [10]:
from matplotlib import pyplot as plt
import seaborn as sns
#sns.distplot(cut['target'])
sns.distplot(y)

In [11]:
from imblearn.over_sampling import SMOTE
smote=SMOTE

In [12]:
import imblearn


In [45]:
from imblearn.over_sampling import SMOTE
#smote=SMOTE(sampling_strategy=0.4,random_state=42)
smote=SMOTE(random_state=42)
X_smote,y_smote=smote.fit_sample(X_train,y_train)
y_smote.value_counts()


1    33721
0    33721
Name: target, dtype: int64

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,roc_auc_score,roc_curve,f1_score
log=LogisticRegression()
log.fit(X_smote,y_smote)
log_predict=log.predict(X_smote)
print('recall:',recall_score(y_smote,log_predict))
print('accuarcy:',accuracy_score(y_smote,log_predict))
print('precision:',precision_score(y_smote,log_predict))


recall: 0.7059102636339373
accuarcy: 0.6902968476616945
precision: 0.6845344222695117


In [49]:
print('                                                   Before SMOTE')
print('accuarcy_score',accuracy_score(y_test,regression_predict))
print('recall',recall_score(y_test,regression_predict))

print('precision:',precision_score(y_test,regression_predict))


print('                                                    After SMOTE')
print('accuarcy:',accuracy_score(y_smote,log_predict))
print('recall:',recall_score(y_smote,log_predict))
print('precision:',precision_score(y_smote,log_predict))




                                                   Before SMOTE
accuarcy_score 0.9630666666666666
recall 0.0
precision: 0.0
                                                    After SMOTE
accuarcy: 0.6902968476616945
recall: 0.7059102636339373
precision: 0.6845344222695117
